# Human Chromosome 21 Complete Synteny Analysis

## Multigenome Synteny Figure (Complete - Including KRTAP Regions)

This notebook generates a **complete synteny figure** for **Human Chromosome 21** vs **Mouse (GRCm39)** and **Rat (GRCr8)**, including all syntenic blocks (KRTAP regions are NOT excluded).

### Data Source
**UCSC Genome Browser REST API** (https://api.genome.ucsc.edu)
- Net alignment tracks: `netMm39`, `netRn7`
- Cytoband ideograms: `cytoBandIdeo`
- Gene annotations: `ncbiRefSeq`

### Requirements
```bash
pip install requests matplotlib numpy
```

### Outputs
- `hsa21_complete_synteny.svg` (vector)
- `hsa21_complete_synteny.png` (300 dpi raster)
- Figure size: **12 × 9 inches** (landscape format)
- All fonts: **Arial** (sizes 12-20pt)

In [9]:
# Import Required Libraries
import sys
import time
import requests
import numpy as np
import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.patches import PathPatch, FancyBboxPatch
from matplotlib.path import Path
from matplotlib.colors import LinearSegmentedColormap

print("All libraries imported successfully!")

All libraries imported successfully!


In [10]:
# Configuration
UCSC_API   = "https://api.genome.ucsc.edu"
OUT_SVG    = "hsa21_complete_synteny.svg"
OUT_PNG    = "hsa21_complete_synteny.png"

# Human chr21 euchromatic region (GRCh38)
HSA21_START =  5_010_000
HSA21_END   = 46_710_000

# Gene lists (all on chr21, all triplicated in trisomy 21)
TIER1_GENES = [          # well-validated DS genes — red triangles above bar
    "APP", "DYRK1A", "RCAN1", "CBS", "SYNJ1",
]
TIER2_GENES = [          # additional chr21 genes of interest — orange triangles below bar
    "BACH1", "IFNAR1", "COL6A1", "TMPRSS2",
]

print("Configuration loaded.")

Configuration loaded.


In [11]:
# Data Fetching Functions

def ucsc_get(endpoint, params=None, retries=3, delay=1.5):
    """GET from UCSC REST API with simple retry logic."""
    url = UCSC_API + endpoint
    for attempt in range(retries):
        try:
            r = requests.get(url, params=params, timeout=60)
            r.raise_for_status()
            return r.json()
        except Exception as e:
            if attempt < retries - 1:
                time.sleep(delay)
            else:
                raise RuntimeError(f"UCSC API failed for {url}: {e}")


def fetch_net_alignments(genome, chrom, track):
    """Fetch top-level net alignment blocks (level==1) for one chromosome."""
    print(f"  Fetching {track} on {genome} {chrom} …")
    data = ucsc_get(f"/getData/track", params={
        "genome": genome, "chrom": chrom, "track": track
    })
    records = data.get(track, [])
    return [r for r in records if r.get("level", 99) == 1]


def fetch_cytobands(genome, chrom):
    print(f"  Fetching cytobands for {genome} {chrom} …")
    data = ucsc_get("/getData/track", params={
        "genome": genome, "chrom": chrom, "track": "cytoBandIdeo"
    })
    return data.get("cytoBandIdeo", [])


def fetch_chrom_sizes(genome):
    print(f"  Fetching chromosome sizes for {genome} …")
    data = ucsc_get("/list/chromosomes", params={"genome": genome})
    return data.get("chromosomes", {})


def fetch_genes(genome, chrom, track="ncbiRefSeq"):
    print(f"  Fetching genes on {genome} {chrom} …")
    data = ucsc_get("/getData/track", params={
        "genome": genome, "chrom": chrom, "track": track
    })
    return data.get(track, [])


def build_gene_lookup(genes):
    """Map gene symbol → {start, end, mid} keeping the longest isoform."""
    lookup = {}
    for g in genes:
        sym = g.get("name2") or g.get("name", "")
        if not sym:
            continue
        s, e = g["txStart"], g["txEnd"]
        if sym not in lookup or (e - s) > (lookup[sym]["end"] - lookup[sym]["start"]):
            lookup[sym] = {"start": s, "end": e, "mid": (s + e) // 2}
    return lookup


def compute_gene_density(genes, chrom_start, chrom_end, bin_size=500_000):
    """Bin gene counts into windows, normalise to [0, 1]."""
    n_bins = (chrom_end - chrom_start) // bin_size + 1
    counts = np.zeros(n_bins)
    for g in genes:
        mid = (g["txStart"] + g["txEnd"]) // 2
        if chrom_start <= mid < chrom_end:
            counts[(mid - chrom_start) // bin_size] += 1
    mx = counts.max()
    return counts / mx if mx > 0 else counts

print("Data fetching functions defined.")

Data fetching functions defined.


In [12]:
# Drawing Helpers and Color Definitions

STAIN_COLORS = {
    "gneg":    "#F2F2F2", "gpos25":  "#C0C0C0", "gpos33":  "#AAAAAA",
    "gpos50":  "#888888", "gpos66":  "#666666", "gpos75":  "#444444",
    "gpos100": "#111111", "acen":    "#C0392B", "gvar":    "#DCDCDC",
    "stalk":   "#BBBBBB",
}

def stain_color(n):
    return STAIN_COLORS.get(n.lower().replace(" ", ""), "#E0E0E0")

CHR_COLORS = {
    "chr10": {"fwd": "#8E44AD", "rev": "#6C3483", "label": "#5B2C6F", "bar": "#D7BDE2"},
    "chr16": {"fwd": "#27AE60", "rev": "#2980B9", "label": "#1A5C2A", "bar": "#A9DFBF"},
    "chr17": {"fwd": "#1ABC9C", "rev": "#117A65", "label": "#0E6655", "bar": "#A2D9CE"},
    "chr11": {"fwd": "#E67E22", "rev": "#C0392B", "label": "#6E2F1A", "bar": "#FAD7A0"},
    "chr20": {"fwd": "#D35400", "rev": "#922B21", "label": "#6E2F1A", "bar": "#F5CBA7"},
}


def draw_chromosome(ax, x0, x1, y_cen, h, bands, genome_start, genome_end,
                    bar_color=None):
    bar_len   = x1 - x0
    chrom_len = genome_end - genome_start
    r         = h * 0.45

    def bp2x(bp):
        return x0 + (bp - genome_start) / chrom_len * bar_len

    y_bot = y_cen - h / 2
    clip  = FancyBboxPatch(
        (x0, y_bot), bar_len, h,
        boxstyle=f"round,pad=0,rounding_size={r}",
        linewidth=0, facecolor="none",
        transform=ax.transData, clip_on=False,
    )
    ax.add_patch(clip)

    if bar_color:
        ax.add_patch(FancyBboxPatch(
            (x0, y_bot), bar_len, h,
            boxstyle=f"round,pad=0,rounding_size={r}",
            facecolor=bar_color, edgecolor="none",
            linewidth=0, clip_on=False, zorder=1,
        ))

    for b in bands:
        bx0 = bp2x(b["chromStart"])
        bx1 = bp2x(b["chromEnd"])
        rect = FancyBboxPatch(
            (bx0, y_bot), bx1 - bx0, h,
            boxstyle="square,pad=0",
            facecolor=stain_color(b.get("gieStain", "gneg")),
            edgecolor="none", linewidth=0,
            clip_on=True, zorder=2,
        )
        rect.set_clip_path(clip)
        ax.add_patch(rect)

    ax.add_patch(FancyBboxPatch(
        (x0, y_bot), bar_len, h,
        boxstyle=f"round,pad=0,rounding_size={r}",
        linewidth=1.5, edgecolor="#333333",
        facecolor="none", clip_on=False, zorder=6,
    ))
    return bp2x


def ribbon(ax, x0a, x1a, ya, x0b, x1b, yb, fc, ec,
           alpha=0.38, lw=0.35, zorder=2):
    cy    = (ya + yb) / 2
    verts = [
        (x0a, ya), (x0a, cy), (x0b, cy), (x0b, yb),
        (x1b, yb), (x1b, cy), (x1a, cy), (x1a, ya), (x0a, ya),
    ]
    codes = [
        Path.MOVETO,
        Path.CURVE4, Path.CURVE4, Path.CURVE4,
        Path.LINETO,
        Path.CURVE4, Path.CURVE4, Path.CURVE4,
        Path.CLOSEPOLY,
    ]
    ax.add_patch(PathPatch(
        Path(verts, codes),
        facecolor=fc, edgecolor=ec,
        alpha=alpha, linewidth=lw,
        zorder=zorder, clip_on=False,
    ))


def assign_levels(gene_list, gene_lookup, n_levels=5, min_sep=2.0e6):
    items = [(gene_lookup[g]["mid"], g) for g in gene_list if g in gene_lookup]
    items.sort()
    occupied = [[] for _ in range(n_levels)]
    result   = []
    for mid, sym in items:
        for lv in range(n_levels):
            if all(abs(mid - pm) >= min_sep for pm in occupied[lv]):
                occupied[lv].append(mid)
                result.append((mid, sym, lv))
                break
        else:
            occupied[-1].append(mid)
            result.append((mid, sym, n_levels - 1))
    return result

print("Drawing helper functions defined.")

Drawing helper functions defined.


In [13]:
# Main Function - Generate Complete Synteny Plot

def main():
    # ── 1. Fetch all data ────────────────────────────────────────────────────
    print("Fetching chromosome sizes …")
    hg38_sizes = fetch_chrom_sizes("hg38")
    mm39_sizes = fetch_chrom_sizes("mm39")
    rn7_sizes  = fetch_chrom_sizes("rn7")

    print("\nFetching cytobands …")
    hsa21_bands = fetch_cytobands("hg38", "chr21")
    mm_bands    = {c: fetch_cytobands("mm39", c) for c in ["chr10", "chr16", "chr17"]}
    rn_bands    = {c: fetch_cytobands("rn7",  c) for c in ["chr11", "chr20"]}

    print("\nFetching net alignments (mouse) …")
    mm_net_raw = fetch_net_alignments("hg38", "chr21", "netMm39")
    # Keep only blocks that map to our three mouse chromosomes
    MM_CHROMS = ["chr10", "chr16", "chr17"]
    mm_top = [b for b in mm_net_raw if b.get("qName") in MM_CHROMS]

    print("\nFetching net alignments (rat) …")
    rn_net_raw = fetch_net_alignments("hg38", "chr21", "netRn7")
    RN_CHROMS  = ["chr11", "chr20"]
    rn_top     = [b for b in rn_net_raw if b.get("qName") in RN_CHROMS]

    # NO KRTAP FILTERING - showing complete synteny
    print(f"  Mouse: {len(mm_top)} synteny blocks (complete, including KRTAP regions)")
    print(f"  Rat: {len(rn_top)} synteny blocks (complete, including KRTAP regions)")

    print("\nFetching RefSeq genes on hg38 chr21 …")
    genes_raw  = fetch_genes("hg38", "chr21")
    gene_lookup = build_gene_lookup(genes_raw)

    BIN_SIZE     = 500_000
    gene_density = compute_gene_density(genes_raw, HSA21_START, HSA21_END, BIN_SIZE)

    # ── 2. Layout constants ──────────────────────────────────────────────────
    CHR_H      = 0.50
    DENS_H     = 0.26
    RIBBON_H   = 2.6
    GAP_H      = 0.18
    LEVEL_STEP = 0.45
    MIN_SEP    = 4.0e6
    N_T2_LV    = 3
    TICK_OFFSET = 0.20 + N_T2_LV * LEVEL_STEP + 0.22

    Y_MOUSE  =  RIBBON_H + GAP_H + CHR_H / 2
    Y_HSA21  =  0.0
    Y_RAT    = -(RIBBON_H + GAP_H + CHR_H / 2)
    HSA21_LEN = HSA21_END - HSA21_START

    # ── 3. Mouse chromosome x-coordinates (proportional widths) ─────────────
    MM_TOTAL = sum(mm39_sizes[c] for c in MM_CHROMS)
    GAP_BP   = int(0.006 * HSA21_LEN)
    mm_x     = {}
    cursor   = HSA21_START
    for i, c in enumerate(MM_CHROMS):
        w = int(mm39_sizes[c] / MM_TOTAL * HSA21_LEN)
        if i < len(MM_CHROMS) - 1:
            w -= GAP_BP // 2
        mm_x[c] = (cursor, cursor + w)
        cursor  += w + GAP_BP
    mm_x["chr17"] = (mm_x["chr17"][0], HSA21_END)   # snap to end

    def mm_bp2x(chrom, bp):
        x0, x1 = mm_x[chrom]
        return x0 + bp / mm39_sizes[chrom] * (x1 - x0)

    # ── 4. Rat chromosome x-coordinates ─────────────────────────────────────
    RN_TOTAL = rn7_sizes["chr11"] + rn7_sizes["chr20"]
    GAP_BP_R = int(0.006 * HSA21_LEN)
    rn11_x0  = HSA21_START
    rn11_x1  = rn11_x0 + int(rn7_sizes["chr11"] / RN_TOTAL * HSA21_LEN) - GAP_BP_R // 2
    rn20_x0  = rn11_x1 + GAP_BP_R
    rn20_x1  = HSA21_END

    def rn11_bp2x(bp): return rn11_x0 + bp / rn7_sizes["chr11"] * (rn11_x1 - rn11_x0)
    def rn20_bp2x(bp): return rn20_x0 + bp / rn7_sizes["chr20"] * (rn20_x1 - rn20_x0)

    # ── 5. Build figure ──────────────────────────────────────────────────────
    print("\nRendering figure …")
    fig, ax = plt.subplots(figsize=(12, 9))
    BG = "#F8F7F2"
    ax.set_facecolor(BG); fig.patch.set_facecolor(BG)
    LEFT_PAD  = 0.10 * HSA21_LEN
    RIGHT_PAD = 0.26 * HSA21_LEN
    ax.set_xlim(HSA21_START - LEFT_PAD, HSA21_END + RIGHT_PAD)
    ax.set_ylim(Y_RAT - 3.4, Y_MOUSE + 2.6)
    ax.axis("off")

    # ── Mouse chromosomes ────────────────────────────────────────────────────
    for c in MM_CHROMS:
        x0, x1 = mm_x[c]
        draw_chromosome(ax, x0, x1, Y_MOUSE, CHR_H,
                        mm_bands[c], 0, mm39_sizes[c],
                        bar_color=CHR_COLORS[c]["bar"])

    # ── Human chr21 + gene density ───────────────────────────────────────────
    draw_chromosome(ax, HSA21_START, HSA21_END, Y_HSA21, CHR_H,
                    hsa21_bands, HSA21_START, HSA21_END)

    dens_y_bot = Y_HSA21 + CHR_H / 2 + 0.05
    dens_y_top = dens_y_bot + DENS_H
    gene_cmap  = LinearSegmentedColormap.from_list("gd", ["#EAF4FB", "#2980B9", "#1A3A5C"])
    for i, d in enumerate(gene_density):
        bx0 = HSA21_START + i * BIN_SIZE
        bx1 = min(bx0 + BIN_SIZE, HSA21_END)
        ax.add_patch(mpatches.Rectangle(
            (bx0, dens_y_bot), bx1 - bx0, DENS_H,
            facecolor=gene_cmap(d), edgecolor="none",
            linewidth=0, zorder=3, clip_on=False,
        ))
    ax.add_patch(mpatches.Rectangle(
        (HSA21_START, dens_y_bot), HSA21_LEN, DENS_H,
        facecolor="none", edgecolor="#AAAAAA", linewidth=0.8,
        zorder=4, clip_on=False,
    ))
    ax.text(HSA21_START - 0.020 * HSA21_LEN, (dens_y_bot + dens_y_top) / 2,
            "Gene\ndensity", ha="right", va="center",
            fontsize=13, fontfamily="Arial", color="#2980B9", clip_on=False)

    # ── Rat chromosomes ──────────────────────────────────────────────────────
    draw_chromosome(ax, rn11_x0, rn11_x1, Y_RAT, CHR_H,
                    rn_bands["chr11"], 0, rn7_sizes["chr11"],
                    bar_color=CHR_COLORS["chr11"]["bar"])
    draw_chromosome(ax, rn20_x0, rn20_x1, Y_RAT, CHR_H,
                    rn_bands["chr20"], 0, rn7_sizes["chr20"],
                    bar_color=CHR_COLORS["chr20"]["bar"])

    # ── Synteny ribbons (COMPLETE - NO FILTERING) ───────────────────────────
    y_hsa_top = Y_HSA21 + CHR_H / 2
    y_hsa_bot = Y_HSA21 - CHR_H / 2
    y_mm_bot  = Y_MOUSE - CHR_H / 2
    y_rn_top  = Y_RAT   + CHR_H / 2

    for b in sorted(mm_top, key=lambda b: b["tEnd"] - b["tStart"], reverse=True):
        hx0, hx1 = b["tStart"], b["tEnd"]
        qchr     = b["qName"]
        qx0 = mm_bp2x(qchr, b["qStart"]); qx1 = mm_bp2x(qchr, b["qEnd"])
        if qx0 > qx1: qx0, qx1 = qx1, qx0
        col = CHR_COLORS[qchr]["fwd"] if b["strand"] == "+" else CHR_COLORS[qchr]["rev"]
        ribbon(ax, qx0, qx1, y_mm_bot, hx0, hx1, y_hsa_top,
               col, CHR_COLORS[qchr]["label"])

    for b in sorted(rn_top, key=lambda b: b["tEnd"] - b["tStart"], reverse=True):
        hx0, hx1 = b["tStart"], b["tEnd"]
        qchr     = b["qName"]
        bp2x     = rn11_bp2x if qchr == "chr11" else rn20_bp2x
        qx0 = bp2x(b["qStart"]); qx1 = bp2x(b["qEnd"])
        if qx0 > qx1: qx0, qx1 = qx1, qx0
        col = CHR_COLORS[qchr]["fwd"] if b["strand"] == "+" else CHR_COLORS[qchr]["rev"]
        ribbon(ax, hx0, hx1, y_hsa_bot, qx0, qx1, y_rn_top,
               col, CHR_COLORS[qchr]["label"])

    # ── Gene markers ─────────────────────────────────────────────────────────
    t1 = assign_levels(TIER1_GENES, gene_lookup, n_levels=5, min_sep=MIN_SEP)
    t2 = assign_levels(TIER2_GENES, gene_lookup, n_levels=N_T2_LV, min_sep=MIN_SEP)

    for mid, sym, lv in t1:
        y_base  = Y_HSA21 + CHR_H / 2
        y_label = y_base + 0.38 + lv * LEVEL_STEP
        ax.plot([mid, mid], [y_base, y_label - 0.06],
                color="#C0392B", lw=0.9, zorder=9, clip_on=False)
        ax.plot(mid, y_base + 0.12, marker="^", ms=7, color="#C0392B",
                zorder=10, clip_on=False,
                markeredgecolor="#7B241C", markeredgewidth=0.5)
        ax.text(mid, y_label, sym,
                ha="center", va="bottom", fontsize=13, fontweight="bold",
                color="#922B21", fontfamily="Arial",
                clip_on=False, zorder=11,
                bbox=dict(facecolor="white", alpha=0.85, edgecolor="none",
                          boxstyle="round,pad=0.12"))

    for mid, sym, lv in t2:
        y_base  = Y_HSA21 - CHR_H / 2
        # Custom adjustments per gene
        if sym in ["BACH1", "COL6A1"]:
            offset = 0.26  # Closer to chr21
        elif sym in ["IFNAR1", "TMPRSS2"]:
            offset = 0.68  # Further from chr21
        else:
            offset = 0.38  # Default
        y_label = y_base - offset - lv * LEVEL_STEP
        ax.plot([mid, mid], [y_base, y_label + 0.06],
                color="#D35400", lw=0.9, zorder=9, clip_on=False)
        ax.plot(mid, y_base - 0.04, marker="v", ms=7, color="#E67E22",
                zorder=10, clip_on=False,
                markeredgecolor="#A04000", markeredgewidth=0.5)
        ax.text(mid, y_label, sym,
                ha="center", va="top", fontsize=13, fontweight="bold",
                color="#A04000", fontfamily="Arial",
                clip_on=False, zorder=11,
                bbox=dict(facecolor="white", alpha=0.85, edgecolor="none",
                          boxstyle="round,pad=0.12"))

    # ── Hsa21 Mb axis ─────────────────────────────────────────────────────────
    tick_y = (Y_HSA21 + Y_RAT) / 2 + 0.15
    for mb in range(10, 47, 5):
        xp = mb * 1_000_000
        if HSA21_START <= xp <= HSA21_END:
            ax.plot([xp, xp], [tick_y, tick_y - 0.15],
                    color="#666", lw=0.9, zorder=7, clip_on=False)
            ax.text(xp, tick_y - 0.20, f"{mb}",
                    ha="center", va="top", fontsize=14, fontfamily="Arial", color="#444", clip_on=False)

    # ── Mouse / rat axis ticks ────────────────────────────────────────────────
    def draw_ticks(bp_fn, x0, x1, y_base, direction, chrom_size_mb, step=20, fs=9.5):
        for mb in range(0, int(chrom_size_mb) + 1, step):
            xp = bp_fn(mb * 1_000_000)
            if x0 <= xp <= x1:
                ax.plot([xp, xp], [y_base, y_base + direction * 0.18],
                        color="#666", lw=0.9, zorder=7, clip_on=False)
                ax.text(xp, y_base + direction * 0.28, f"{mb}",
                        ha="center",
                        va="bottom" if direction > 0 else "top",
                        fontsize=fs, fontfamily="Arial", color="#555", clip_on=False)

    for c in MM_CHROMS:
        x0, x1 = mm_x[c]
        draw_ticks(lambda bp, c=c: mm_bp2x(c, bp),
                   x0, x1, Y_MOUSE + CHR_H / 2, +1, mm39_sizes[c] / 1e6, step=20, fs=12)
    draw_ticks(rn11_bp2x, rn11_x0, rn11_x1, Y_RAT - CHR_H / 2, -1, rn7_sizes["chr11"] / 1e6, step=20, fs=12)
    draw_ticks(rn20_bp2x, rn20_x0, rn20_x1, Y_RAT - CHR_H / 2, -1, rn7_sizes["chr20"] / 1e6, step=20, fs=12)

    # ── Chromosome labels ─────────────────────────────────────────────────────
    def chr_label(x0, x1, y, text, color, fs=14):
        ax.text((x0 + x1) / 2, y, text,
                ha="center", va="center", fontsize=fs, fontweight="bold",
                color=color, fontfamily="Arial",
                clip_on=False, zorder=8,
                bbox=dict(facecolor="white", alpha=0.60, edgecolor="none",
                          boxstyle="round,pad=0.18"))

    for c in MM_CHROMS:
        x0, x1 = mm_x[c]
        chr_label(x0, x1, Y_MOUSE, f"chr {c[3:]}", CHR_COLORS[c]["label"], fs=16)
    chr_label(HSA21_START, HSA21_END, Y_HSA21, "chr 21", "#1A1A6E", fs=18)
    chr_label(rn11_x0, rn11_x1, Y_RAT, "chr 11", CHR_COLORS["chr11"]["label"], fs=16)
    chr_label(rn20_x0, rn20_x1, Y_RAT, "chr 20", CHR_COLORS["chr20"]["label"], fs=16)

    # ── Species labels ────────────────────────────────────────────────────────
    sp_x = HSA21_START - 0.040 * HSA21_LEN

    def sp_label(x, y, name, asm, nc, ac, shift_down=False):
        if shift_down:
            ax.text(x, y - 0.55, name, ha="right", va="top", fontsize=17,
                    style="italic", fontweight="bold", color=nc,
                    fontfamily="Arial", clip_on=False)
            ax.text(x, y - 0.80, asm, ha="right", va="top", fontsize=13,
                    color=ac, fontfamily="Arial", clip_on=False)
        else:
            ax.text(x, y + 0.25, name, ha="right", va="bottom", fontsize=17,
                    style="italic", fontweight="bold", color=nc,
                    fontfamily="Arial", clip_on=False)
            ax.text(x, y - 0.25, asm, ha="right", va="top", fontsize=13,
                    color=ac, fontfamily="Arial", clip_on=False)

    sp_label(sp_x, Y_MOUSE, "Mus musculus",     "GRCm39", "#1A5C2A", "#2E7D32")
    sp_label(sp_x, Y_HSA21, "Homo sapiens",      "GRCh38", "#1A1A6E", "#283593", shift_down=True)
    sp_label(sp_x, Y_RAT,   "Rattus norvegicus", "GRCr8",  "#7D3C00", "#A04000")

    for y_div in [(Y_MOUSE + Y_HSA21) / 2, (Y_HSA21 + Y_RAT) / 2]:
        ax.axhline(y_div, xmin=0.085, xmax=0.775,
                   color="#CCCCCC", lw=0.8, ls="--", zorder=0, clip_on=False)

    # ── Legend ────────────────────────────────────────────────────────────────
    leg_x = HSA21_END + 0.065 * HSA21_LEN
    leg_y = Y_MOUSE + 0.8
    row_h = 0.40
    sw    = 0.015 * HSA21_LEN

    def leg_section(x, y, title):
        ax.text(x, y, title, ha="left", va="bottom", fontsize=15,
                fontweight="bold", color="#222",
                fontfamily="Arial", clip_on=False)
        return y - 0.16

    def leg_item(x, y, col, label, alpha=0.85, ec="#555"):
        ax.add_patch(FancyBboxPatch(
            (x, y - 0.11), sw, 0.22, boxstyle="round,pad=0.01",
            facecolor=col, edgecolor=ec, linewidth=0.8,
            alpha=alpha, clip_on=False,
        ))
        ax.text(x + sw + 0.004 * HSA21_LEN, y, label,
                ha="left", va="center", fontsize=13, color="#333",
                fontfamily="Arial", clip_on=False)

    leg_y = leg_section(leg_x, leg_y, "Synteny Orientation")
    for col, lbl in [
        (CHR_COLORS["chr10"]["fwd"], "Mouse chr 10  (+)"),
        (CHR_COLORS["chr10"]["rev"], "Mouse chr 10  (−)"),
        (CHR_COLORS["chr16"]["fwd"], "Mouse chr 16  (+)"),
        (CHR_COLORS["chr16"]["rev"], "Mouse chr 16  (−)"),
        (CHR_COLORS["chr17"]["fwd"], "Mouse chr 17  (+)"),
        (CHR_COLORS["chr17"]["rev"], "Mouse chr 17  (−)"),
    ]:
        leg_y -= row_h; leg_item(leg_x, leg_y, col, lbl)

    leg_y -= row_h * 0.3
    for col, lbl in [
        (CHR_COLORS["chr11"]["fwd"], "Rat chr 11  (+)"),
        (CHR_COLORS["chr11"]["rev"], "Rat chr 11  (−)"),
        (CHR_COLORS["chr20"]["fwd"], "Rat chr 20  (+)"),
        (CHR_COLORS["chr20"]["rev"], "Rat chr 20  (−)"),
    ]:
        leg_y -= row_h; leg_item(leg_x, leg_y, col, lbl)

    leg_y -= row_h * 1.6
    leg_y  = leg_section(leg_x, leg_y, "Chr 21 Gene Markers")
    leg_y -= row_h * 0.4
    ax.plot(leg_x + sw / 2, leg_y, marker="^", ms=8, color="#C0392B",
            clip_on=False, markeredgecolor="#7B241C", markeredgewidth=0.6)
    ax.text(leg_x + sw + 0.004 * HSA21_LEN, leg_y, "Well-validated DS gene",
            ha="left", va="center", fontsize=13, color="#922B21",
            fontweight="bold", fontfamily="Arial", clip_on=False)
    leg_y -= row_h
    ax.plot(leg_x + sw / 2, leg_y, marker="v", ms=8, color="#E67E22",
            clip_on=False, markeredgecolor="#A04000", markeredgewidth=0.6)
    ax.text(leg_x + sw + 0.004 * HSA21_LEN, leg_y, "Additional chr 21 gene",
            ha="left", va="center", fontsize=13, color="#A04000",
            fontweight="bold", fontfamily="Arial", clip_on=False)

    # ── Title ─────────────────────────────────────────────────────────────────
    title_x = (HSA21_START + HSA21_END) / 2
    ax.text(title_x, Y_MOUSE + 2.00,
            "Complete Syntenic Regions of Human Chromosome 21",
            ha="center", va="bottom", fontsize=20, fontweight="bold",
            color="#111111", fontfamily="Arial", clip_on=False)

    # ── Save ──────────────────────────────────────────────────────────────────
    print(f"\nSaving {OUT_SVG} …")
    fig.savefig(OUT_SVG, format="svg", dpi=300, bbox_inches="tight",
                facecolor=fig.get_facecolor())
    print(f"Saving {OUT_PNG} …")
    fig.savefig(OUT_PNG, format="png", dpi=300, bbox_inches="tight",
                facecolor=fig.get_facecolor())
    plt.close()
    print("Done.")

print("Main function defined.")

Main function defined.


In [14]:
# Run the analysis
# This will fetch data from UCSC API and generate the complete synteny plot
# Output files: hsa21_complete_synteny.svg and hsa21_complete_synteny.png

main()

Fetching chromosome sizes …
  Fetching chromosome sizes for hg38 …


  Fetching chromosome sizes for mm39 …
  Fetching chromosome sizes for rn7 …

Fetching cytobands …
  Fetching cytobands for hg38 chr21 …
  Fetching cytobands for mm39 chr10 …
  Fetching cytobands for mm39 chr16 …
  Fetching cytobands for mm39 chr17 …
  Fetching cytobands for rn7 chr11 …
  Fetching cytobands for rn7 chr20 …

Fetching net alignments (mouse) …
  Fetching netMm39 on hg38 chr21 …

Fetching net alignments (rat) …
  Fetching netRn7 on hg38 chr21 …
  Mouse: 63 synteny blocks (complete, including KRTAP regions)
  Rat: 37 synteny blocks (complete, including KRTAP regions)

Fetching RefSeq genes on hg38 chr21 …
  Fetching genes on hg38 chr21 …

Rendering figure …

Saving hsa21_complete_synteny.svg …
Saving hsa21_complete_synteny.png …
Done.
